# NBA Data Wrangling

## Introduction

My capstone project is based on the NBA statistics and specifically, to come up with a better predictor for points than the Y-T-D average or last year's average. The purpose of this would be to be implemented on a client-facing Fantasy Basketball website to show what your team's points will be.

As mentioned in the proposal, I will be gathering data directly from stats.nba.com - the official source of data for NBA statistics. More over, a group of programmers have put together the nba_py library - a Python facing API for stats.nba.com.

## API Exploration
#### Summary
After looking at the various documentations and other coder's codes, I have narrowed down my data sources to come from two modules in the nba_py source: PlayerStats and GameLog. On top of this, we will be depending on several Pandas features to merge the datasets together.

The NBA has been keeping a track of these statistics ever since the 1970's. However, we only care about recent years data so we can optimize our machine learning algorithmns to adjust for recent year's styles of play - especially since the recent rise of the 3 point shot/perimeter spreading. I will be take the last 4 years of data, with the first 3 years being my training set and my last year being my test set.

In [5]:
from IPython.display import Math
import pandas as pd
from nba_py.league import PlayerStats, GameLog

seasons = ['2014-15', '2015-16', '2016-17', '2017-18']

## API Exploration
#### PlayerStats Module

The PlayerStats module gives us the player's performance for the entire year. We will be using this to clean our data per the below reasons/steps.

1. Player statistics can get skewed by "garbage" time players (players who only play during the last few minutes of the game when the game is not closely decided) and can get skewed by players with limited playing time.
2. We can then limit the players to only players who play 30 games or more, and have played for more than 15 minutes to ensure we are doing our analysis solely on significant minutes players.
3. The GameLog module does not keep a track of each player's position. Thus we will need to use the PlayerStats module to get the listsed position of each player.

Now, let's show a snippet of what the data looks like for the 2017-18 season.

In [3]:
PlayerStats(season='2017-18').overall().head()

,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,MIN,...,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,CFID,CFPARAMS
0,201166,Aaron Brooks,1610612750,MIN,33.0,32,19,13,0.594,5.9,...,427,447,472,459,422,484,228,32,5,"201166,1610612750"
1,203932,Aaron Gordon,1610612753,ORL,22.0,58,19,39,0.328,32.9,...,78,197,49,43,377,43,30,32,5,"203932,1610612753"
2,1626151,Aaron Harrison,1610612742,DAL,23.0,9,2,7,0.222,25.9,...,485,16,229,274,534,269,228,32,5,"1626151,1610612742"
3,1628935,Aaron Jackson,1610612745,HOU,32.0,1,0,1,0.000,34.5,...,25,1,58,223,538,334,228,32,5,"1628935,1610612745"
4,1627846,Abdel Nader,1610612738,BOS,24.0,48,33,15,0.688,10.9,...,348,439,420,432,418,447,228,32,5,"1627846,1610612738"


## API Exploration
#### GameLog Module

This is the meat of where our data used for analysis will be. This is a gamelog that tracks how every player or team performed for every game they played throughout the season. We can define if we want to look at individual players or by team. Both will be important in our analysis.

1. Individual player statistic will obviously be important because past game performance will be the bulk data of how a player performs. We'll keep all individual game per player performance.

2. Within a game, both team's PACE of play is also important as this can be used to estimate how many possessions will be in the game. A logical hypothesis can be made that a lower PACE game can result in all players having lower than average points with all things being equal. This will of course have to be shown in the data exploration phase, but this can indicate the importance of PACE. PACE has several ways defined. We will be using the Basketball-Reference's definition.

The below is the formula used to define PACE. The glossary of acronyms for the PACE is:
1. FGA - Field Goals Attempted
2. FGM - Field Goals Made
3. FTA - Free Throws Attempted
4. OREB - Team's Offensive Rebound
5. OPPDREB - Opponent's Defensive Rebound
6. TOV - Turnovers

In [16]:
Math(r'PACE = FGA + 0.4 * FTA - 1.07  (FGA-FGM)\frac{OREB}{OREB + OPPDREB} + TOV')

<IPython.core.display.Math object>

Now, let's show a snippet of what the GameLog data looks like for the 2017-18 season.

In [18]:
GameLog(season='2017-18',player_or_team='T').overall().head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22017,1610612748,MIA,Miami Heat,0021701054,2018-03-19,MIA vs. DEN,W,290,56,...,39,46,36,5,11,11,26,149,8,1
1,22017,1610612760,OKC,Oklahoma City Thunder,0021700674,2018-01-20,OKC @ CLE,W,240,58,...,36,52,37,9,2,14,23,148,24,1
2,22017,1610612745,HOU,Houston Rockets,0021700540,2017-12-31,HOU vs. LAL,W,290,49,...,28,43,29,7,9,12,33,148,6,1
3,22017,1610612743,DEN,Denver Nuggets,0021700226,2017-11-17,DEN vs. NOP,W,240,56,...,35,48,37,5,4,15,15,146,32,1
4,22017,1610612744,GSW,Golden State Warriors,0021700276,2017-11-24,GSW vs. CHI,W,240,53,...,43,49,36,11,11,11,15,143,49,1


In [20]:
GameLog(season='2017-18',player_or_team='P').overall().head()

,SEASON_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22017,201935,James Harden,1610612745,HOU,Houston Rockets,0021700748,2018-01-30,HOU vs. ORL,W,...,8,10,11,4,1,5,2,60,7,1
1,22017,2544,LeBron James,1610612739,CLE,Cleveland Cavaliers,0021700122,2017-11-03,CLE @ WAS,W,...,6,11,7,3,2,3,2,57,6,1
2,22017,1626157,Karl-Anthony Towns,1610612750,MIN,Minnesota Timberwolves,0021701120,2018-03-28,MIN vs. ATL,W,...,11,15,4,0,1,2,4,56,7,1
3,22017,201935,James Harden,1610612745,HOU,Houston Rockets,0021700140,2017-11-05,HOU vs. UTA,W,...,2,2,13,2,0,7,3,56,22,1
4,22017,203076,Anthony Davis,1610612740,NOP,New Orleans Pelicans,0021700902,2018-02-26,NOP vs. PHX,W,...,10,18,3,1,5,1,3,53,9,1


## Assumptions in the Creating our Analysis Data

Now, it is important to note that this is our raw data. We can't use this data without first cleaning the data as the results can have forecast biasness in it if we are using the same Player's Game data to predict a player's Points. We'll need to clean the data and have a few assumptions in our data. The rest of this Jupyter Library will not have any markdown in it. I will be defining a function to combine these sets of data with our assumptions that I will list below to gather our 4 years of data. I will keep the function well documented while I work through my thoughts. After we create our dataframe for analysis, I will save them to CSV files for later analysis/machine learning applications.

Assumptions:

1. As mentioned, in the above PlayerStats module description, we don't want to take into account players who player a non-significant amount of time/not enough games. Thus we will be limiting the players players who play more than 15 minutes and have played more than 30 games.

2. We will be using a rolling average for each player's individual statistics. The rolling window will be 3 games. This will mean that we will be dropping each players first three games to build the model. **This window may change depending on statistical significance. We may change this to every 5 games or a YTD. **

3. As mentioned, we will be using the BasketBall Reference's definition of PACE. We will also be calculating the overall PACE and then doing a Y-T-D average on it. **This may change to take a Y-T-D average of every variable in the formula and then applying the formula.**

In [30]:
#Helper Function used to define PACE (will be used in cleaning_data function)
def pace_calculator(FGA, FTA, OREB, OPPDREB, FG, TOV, MIN):
    return 240 * (FGA + 0.4 * FTA - 1.07 * (FGA-FG)* (OREB/(OREB + OPPDREB)) + TOV) / MIN

In [34]:
#Helper Function to remove the first three rows of data (will be used to remove the first three rows of data of each player - mentioned above assumption #2)
def remove_3(df):
    return df.iloc[3:]

In [45]:
def cleaning_data(year):
    #Defining all dataframes from NBA_py
    team_games = GameLog(season=year,player_or_team='T').overall()
    player_games = GameLog(season=year,player_or_team='P').overall()
    player_overall_df = PlayerStats(season=year).overall()
    
    #Filtering out players in player_games that do not bear significance in this analysis
    player_overall_df = player_overall_df[player_overall_df['MIN'] > 15]
    player_overall_df = player_overall_df[player_overall_df['GP'] > 30]
    players = player_overall_df['PLAYER_ID'].tolist()
    player_games = player_games[player_games['PLAYER_ID'].isin(players)]
    
    #Sorting values to be each player and games played (earliest to latest)
    player_games = player_games.sort_values(['PLAYER_ID','GAME_ID']).set_index(['GAME_ID'])
    
    #Creating dictionary of new column names for after a mean is applied
    lis = ['MIN','FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA', 'OREB', 'DREB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']
    lis_modified = [item + '_AVGLAST3GAMES' for item in lis]
    dictionary = dict(zip(lis, lis_modified))
    
    #Group Players and then apply the rolling mean of 3
    player_games_grouped = player_games.groupby(['PLAYER_ID'])
    player_games_mean = pd.DataFrame(player_games_grouped.rolling(center=False,window=3,win_type='triang')['MIN','FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA', 'OREB', 'DREB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'].mean().shift()).rename(index=str, columns=dictionary).reset_index()
    #Grab necessary columns
    player_games_cleaned = player_games[['SEASON_ID', 'PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID',
       'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_DATE', 'MATCHUP',
       'WL', 'PTS']].reset_index()
    #PLAYER_ID got switched to integer, switching it back to a string for merge purposes.
    player_games_cleaned['PLAYER_ID'] = player_games_cleaned['PLAYER_ID'].astype(str)
    #Merging the original necessary columns dataframe (i.e. the non-statistical valued columns) with our mean valued dataframes.
    
                                                      
    player_data = pd.merge(player_games_cleaned, player_games_mean, how="inner", left_on=['PLAYER_ID', 'GAME_ID'], right_on=['PLAYER_ID', 'GAME_ID'])
    player_data_temp = player_data.set_index(['GAME_ID'], drop= False)
    
    
    player_pts_ytd = player_data_temp.groupby('PLAYER_ID').PTS.expanding(min_periods=2).mean().reset_index()
    player_data_merged = pd.merge(player_data, player_pts_ytd, how="inner", left_on=['PLAYER_ID', 'GAME_ID'], right_on=['PLAYER_ID', 'GAME_ID']).rename(index=str, columns={"PTS_y": "PTS_YTD"})
    player_data_cleaned = player_data_merged.set_index(['PLAYER_ID','GAME_ID']).groupby(level=0, group_keys=False).apply(remove_3)
    player_data_cleaned = player_data_cleaned.reset_index()
    #Values in the team game only have the specific team's data (does not have the opponent's data), thus we need to append it.
    team_games = team_games.sort_values(['TEAM_ID','GAME_ID']).set_index(['GAME_ID'], drop=False)
    team_games['VS_TEAM_ID'] = team_games['MATCHUP'].str[-3:]
    #creating the vs team's columns
    cols = ['SEASON_ID','GAME_DATE', 'MATCHUP','TEAM_ABBREVIATION' ]
    cols2 = ['MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS']
    for item in cols2:
        cols.append('VS_TEAM_'+item)
    temp_df = team_games[['SEASON_ID','GAME_DATE', 'MATCHUP','TEAM_ABBREVIATION', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS']]    
    temp_df.columns = cols
    #merging the two columns together to create merged_tg which will have both the specified team's data as well as the opponent's data
    merged_tg = pd.merge(team_games, temp_df, how='inner', left_on=['VS_TEAM_ID', 'GAME_DATE'], right_on=['TEAM_ABBREVIATION', 'GAME_DATE'])
    #We can apply our pace_calculator helper now
    merged_tg['PACE'] =  pace_calculator(FGA=merged_tg['FGA'], FTA=merged_tg['FTA'],OREB=merged_tg['OREB'], OPPDREB=merged_tg['VS_TEAM_DREB'], FG=merged_tg['FGM'], TOV=merged_tg['TOV'], MIN= merged_tg['MIN'] )
    merged_tg =merged_tg.set_index(['GAME_ID'], drop= False)
    #Applying the Y-T-D mean
    expander = merged_tg.groupby('TEAM_ABBREVIATION_x').PACE.expanding(min_periods=2).mean().reset_index()
    #Merging this Y-T-D PACE mean with other team data
    temp_team_df = pd.merge(left = merged_tg[['TEAM_NAME','TEAM_ABBREVIATION_x','GAME_ID', 'GAME_DATE','PTS', 'TEAM_ABBREVIATION_y', 'VS_TEAM_PTS']], right= expander, how="inner", left_on=['TEAM_ABBREVIATION_x', 'GAME_ID'], right_on=['TEAM_ABBREVIATION_x','GAME_ID'])
    final_team_df = pd.merge(left = temp_team_df, right= expander, how="inner", left_on=['TEAM_ABBREVIATION_y', 'GAME_ID'], right_on=['TEAM_ABBREVIATION_x','GAME_ID']).rename(index=str, columns={"PACE_x": "OWN_TEAM_PACE_YTD","PACE_y": "VS_TEAM_PACE_YTD", "PTS_x":"PTS_ACTUAL"})
    #Merge together with player_data_cleaned
    merged_player_data = pd.merge(player_data_cleaned, final_team_df, how="inner", left_on=['TEAM_NAME', 'GAME_ID'], right_on=['TEAM_NAME', 'GAME_ID'])
    merged_player_data = merged_player_data.sort_values(['PLAYER_ID', 'GAME_ID']).rename(index=str, columns={"PTS": "OWN_TEAM_PTS","TEAM_ABBREVIATION_y": "VS_TEAM_ABBREVIATION", "PTS_x":"PTS_ACTUAL"})
    merged_player_data = merged_player_data.drop(['GAME_DATE_y','TEAM_ABBREVIATION_x_x','TEAM_ABBREVIATION_x_y'],axis=1)
    return merged_player_data

In [46]:
cleaning_data('2017-18').head()

,PLAYER_ID,GAME_ID,SEASON_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_DATE_x,MATCHUP,WL,...,BLK_AVGLAST3GAMES,TOV_AVGLAST3GAMES,PF_AVGLAST3GAMES,PTS_AVGLAST3GAMES,PTS_YTD,OWN_TEAM_PTS,VS_TEAM_ABBREVIATION,VS_TEAM_PTS,OWN_TEAM_PACE_YTD,VS_TEAM_PACE_YTD
0,101107,0021700055,22017,Marvin Williams,1610612766,CHA,Charlotte Hornets,2017-10-25,CHA vs. DEN,W,...,0.333333,0.666667,1.666667,5.333333,6.250000,110,DEN,93,98.427838,95.974228
6,101107,0021700070,22017,Marvin Williams,1610612766,CHA,Charlotte Hornets,2017-10-27,CHA vs. HOU,L,...,0.333333,1.000000,1.666667,6.000000,5.600000,93,HOU,109,98.028270,97.588361
11,101107,0021700088,22017,Marvin Williams,1610612766,CHA,Charlotte Hornets,2017-10-29,CHA vs. ORL,W,...,0.000000,1.000000,2.000000,7.000000,6.833333,120,ORL,113,99.581654,103.397262
16,101107,0021700096,22017,Marvin Williams,1610612766,CHA,Charlotte Hornets,2017-10-30,CHA @ MEM,W,...,0.333333,0.666667,1.333333,8.333333,7.142857,104,MEM,99,99.750232,95.313199
23,101107,0021700105,22017,Marvin Williams,1610612766,CHA,Charlotte Hornets,2017-11-01,CHA vs. MIL,W,...,0.666667,0.666667,1.000000,8.333333,8.125000,126,MIL,121,99.645565,97.265735


## Save the CSV files

In [47]:
for year in seasons:
    df = cleaning_data(year)
    df.to_csv(path_or_buf="cleaned_data"+year+".csv")